# Data Wrangling & Baseline Model
---

#### Importing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# import nltk
from sklearn import *

In [2]:
import matplotlib.style as style 

style.use('seaborn-darkgrid')
sns.set_context('notebook')
sns.set_palette('gist_heat')

## Wrangling

In [3]:
os.listdir('./csv_folder')

['sales_train.csv',
 'shops.csv',
 'test.csv',
 'item_categories.csv',
 '.ipynb_checkpoints',
 'items.csv']

- We will need to join the ***items*** and the ***sales_train*** dataframes
- The ***shops*** and ***item_categories*** only contain the names of id values
- The final file ***test*** will be used when submitting predictions

In [4]:
# shops = pd.read_csv('./csv_folder/shops.csv')
# shops.head()

The names of the shops are in russian, should find some encoding that is capable of handling these

In [5]:
# item_categs = pd.read_csv('./csv_folder/item_categories.csv')
# item_categs.head()

As shown above, the ***shops*** and ***item_categories*** only contain the names of id values. These may be useful later but for now we will use only the id values.

In [6]:
# test = pd.read_csv('./csv_folder/test.csv')
# test.head()

Showing the ***test*** data for reference

---
---  
### Understanding the sales_train data
---

In [7]:
train_df = pd.read_csv('./csv_folder/sales_train.csv')

In [8]:
# sales_train

In [9]:
# sales_train.info()

In [10]:
# sales_train['date'] = pd.to_datetime(sales_train['date'], dayfirst=True)
# sales_train.info()

Lets break the date column into month and year, later we will aggregate item sales counts from days to months

In [11]:
# sales_train.describe()

In [12]:
# sales_train['date_block_num'].unique()

***date_block_num***: a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

In [13]:
# fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

# sns.distplot(sales_train['date_block_num'], ax=axes[0,0]).set_title('date_block_num', size=22)

# sns.distplot(sales_train['item_price'], ax=axes[0,1]).set_title('item_price', size=22)

# sns.distplot(sales_train['item_cnt_day'], ax=axes[1,0]).set_title('item_cnt_day', size=22)

# axes[1,1].set_visible(False)

# plt.tight_layout()

In [14]:
# sales_train['item_cnt_day'].nunique()

In [15]:
# sales_train.index.nunique()

In [16]:
# sales_train.isnull().sum()

Dataframe contains no missing values

---
---  
### Understanding the items data
---

In [17]:
# items = pd.read_csv('./csv_folder/items.csv')

In [18]:
# items

In [19]:
# items['item_category_id'].nunique()

There are 84 different categories for items

In [20]:
# items.isnull().sum()

Dataframe contains no missing values

---
---  
### Merging the sales train data frame with the items data 
---

In [21]:
# train_df = pd.merge(sales_train, items.drop(columns=['item_name']), how='left', on='item_id')

In [22]:
train_df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [23]:
train_df.pivot_table(index = ['shop_id','item_id'],
                     values = ['item_cnt_day'],
                     columns = ['date_block_num'],
                     fill_value = 0,
                     aggfunc='sum')

item_cnt_day                              ...                 \
date_block_num            0   1  2  3  4  5  6  7  8  9   ... 24 25 26 27 28   
shop_id item_id                                           ...                  
0       30                 0  31  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        31                 0  11  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        32                 6  10  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        33                 3   3  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        35                 1  14  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
...                      ...  .. .. .. .. .. .. .. .. ..  ... .. .. .. .. ..   
59      22154              1   0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        22155              0   0  0  0  0  0  1  0  0  0  ...  0  0  0  0  0   
        22162              0   0  0  0  0  0  0  0  0  0  ...  0  9  4  1  1   
        22164              0   0  0  0  0  0  0  0  0  0  ...  0  2  1  2  0   
        22167              0   0  0  0  0  0  0  0  0  1  ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
0       30       0  0  0  0  0  
        31       0  0  0  0  0  
        32       0  0  0  0  0  
        33       0  0  0  0  0  
        35       0  0  0  0  0  
...             .. .. .. .. ..  
59      22154    0  0  0  0  0  
        22155    0  0  0  0  0  
        22162    0  0  1  0  0  
        22164    0  1  0  0  0  
        22167    0  0  0  0  0  

[424124 rows x 34 columns]

***A pivot table to give a better visual understanding of the data***

---  
---
### Some Feature Engineering
---  

In [24]:
train_df.drop(['item_price'], axis=1, inplace=True)

In [25]:
train_df = train_df.groupby([c for c in train_df.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train_df = train_df.rename(columns={'item_cnt_day':'item_cnt_month'})

Aggregating the item sales count into months since that is the figure we are being asked to predict

In [26]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_cnt_month
0,01.01.2013,0,2,991,1.0
1,01.01.2013,0,2,1472,1.0
2,01.01.2013,0,2,1905,1.0
3,01.01.2013,0,2,2920,2.0
4,01.01.2013,0,2,3320,1.0


***Filling missing rows***  
Creating rows for days that were not recorded due to zero sales of an item at a specific shop, then filling item count month features na values to be zero

In [27]:
train_df['date'] = pd.to_datetime(train_df['date'], dayfirst=True)

In [28]:
# train_df['month'] = train_df['date'].dt.month
# train_df['year'] = train_df['date'].dt.year

In [29]:
shop_items = train_df[['shop_id', 'item_id']].values
year = train_df.date.dt.year.unique()
month = train_df.date.dt.month.unique()
dates = [(y, m) for y in year for m in month]

index = pd.DataFrame([(d[0], d[1], k[0], k[1]) for d in dates for k in shop_items], 
                     columns=['year', 'month', 'shop_id', 'item_id'])
index['date'] = pd.to_datetime(index[['year', 'month']].assign(DAY=1))
index.drop(columns=['year', 'month'], inplace=True)

In [34]:
index['shop_id'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

In [35]:
# train_df['date'] = pd.to_datetime(train_df[['year', 'month']].assign(DAY=1))
train_df = pd.merge(index, train_df, 
                    how='left', 
                    on=['date', 'shop_id', 'item_id'])

In [ ]:
train_df.to_csv('wrangled_data/traindf_all_index.csv', index=False)

In [39]:
train_df.sort_values(by='shop_id', ascending=True).head()

,shop_id,item_id,date,date_block_num,item_cnt_month
62541327,0,3318,2014-10-01,NaN,NaN
6168720,0,21572,2013-03-01,NaN,NaN
6168719,0,21566,2013-03-01,NaN,NaN
6168718,0,21552,2013-03-01,NaN,NaN
6168717,0,21462,2013-03-01,NaN,NaN


In [41]:
train_df['item_cnt_month'].fillna(0, inplace=True)

In [42]:
train_df.isna().sum()

shop_id                   0
item_id                   0
date                      0
date_block_num    101672860
item_cnt_month            0
dtype: int64

In [44]:
index = index.set_index(['shop_id', 'item_id'])
index.sort_values(by=['shop_id', 'item_id'], ascending=True, inplace=True)

In [47]:
train_df = train_df.set_index(['shop_id', 'item_id'])

In [48]:
del index

In [52]:
train_df.drop((train_df.groupby(['shop_id', 'item_id'])['item_cnt_month'].sum() == 0).index, inplace=True)
# (train_df.groupby(['shop_id', 'item_id'])['item_cnt_month'].sum() == 0).index

In [56]:
train_df.head()

,,date,date_block_num,item_cnt_month
shop_id,item_id,,,


In [82]:
train_df['date_block_num'].fillna(key_df.loc[train_df.date.dt.year, train_df.date.dt.month].values[0])

NotImplementedError: Index._join_level on non-unique index is not implemented

In [96]:
train_df

,shop_id,item_id,date,date_block_num,month,year,item_category_id,item_cnt_month
0,0,32,2013-01-01,0.0,1.0,2013.0,40.0,6.0
1,0,33,2013-01-01,0.0,1.0,2013.0,37.0,3.0
2,0,35,2013-01-01,0.0,1.0,2013.0,40.0,1.0
3,0,43,2013-01-01,0.0,1.0,2013.0,40.0,1.0
4,0,51,2013-01-01,0.0,1.0,2013.0,57.0,2.0
...,...,...,...,...,...,...,...,...
57928459,59,22087,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928460,59,22088,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928461,59,22091,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928462,59,22100,2015-12-01,NaN,NaN,NaN,NaN,NaN


In [100]:
train_df.shape

(57928464, 8)

In [101]:
nulldf = train_df[train_df['date_block_num'].isna()]
nulldf

,shop_id,item_id,date,date_block_num,month,year,item_category_id,item_cnt_month
63224,0,30,2013-01-01,NaN,NaN,NaN,NaN,NaN
63225,0,31,2013-01-01,NaN,NaN,NaN,NaN,NaN
63229,0,36,2013-01-01,NaN,NaN,NaN,NaN,NaN
63230,0,40,2013-01-01,NaN,NaN,NaN,NaN,NaN
63231,0,42,2013-01-01,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
57928459,59,22087,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928460,59,22088,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928461,59,22091,2015-12-01,NaN,NaN,NaN,NaN,NaN
57928462,59,22100,2015-12-01,NaN,NaN,NaN,NaN,NaN


In [ ]:
key_df = train_df[['date_block_num', 'year', 'month']].groupby(['date_block_num', 'year', 'month']).count().reset_index().set_index(['year', 'month'])

In [265]:
tmp = train_df[train_df['date_block_num'].isna()]

In [268]:
tmp.head()

,shop_id,item_id,date,date_block_num,month,year,item_category_id,item_cnt_month
63224,0,30,2013-01-01,NaN,NaN,NaN,NaN,NaN
63225,0,31,2013-01-01,NaN,NaN,NaN,NaN,NaN
63229,0,36,2013-01-01,NaN,NaN,NaN,NaN,NaN
63230,0,40,2013-01-01,NaN,NaN,NaN,NaN,NaN
63231,0,42,2013-01-01,NaN,NaN,NaN,NaN,NaN


In [269]:
# train_df[train_df.isna()]
# train_df[train_df['date_block_num'].isna()].apply(lambda x: train.date.dt.year if x.name == 'year' else 0, axis=1)
null = tmp.copy()
def fillmissing (row):
    row['year'] = row['date'].year
    row['month'] = row['date'].month
    row['date_block_num'] = key_df.loc[row['date'].year, row['date'].month].values[0]
    return row
#     return row['shop_id'],row['item_id'], row['date'],row['date_block_num'], row['year'],row['month'], row['item_category_id'], row['item_cnt_month']
null = tmp.apply(lambda row: fillmissing(row), axis=1)

# train_df[train_df['date_block_num'].isna()].apply(lambda x: np.square(x) if x.name == 'date_block_num', axis=1)
# train_df[lambda train_df:]

KeyError: 11

In [ ]:
pd.DataFrame(null)

In [109]:
train_df['date'][0].dt.year

AttributeError: 'Timestamp' object has no attribute 'dt'

In [75]:
key_df = train_df[['date_block_num', 'year', 'month']].groupby(['date_block_num', 'year', 'month']).count().reset_index().set_index(['year', 'month'])

# train_df['date_block_num'] = 
key_df.loc[train_df.date.year, train_df.date.month].values[0]

# train_df.isna().sum()

AttributeError: 'Series' object has no attribute 'year'

In [ ]:
key_df = train_df[['date_block_num', 'year', 'month']].groupby(['date_block_num', 'year', 'month']).count().reset_index().set_index(['year', 'month'])

train_df['date_block_num'] = key_df.loc[train_df.date.year, train_df.date.month].values[0]
train_df['year'] = train_df['year'].fillna(train_df.date.year)
train_df['month'] = train_df['month'].fillna(train_df.date.month)
train_df['item_cnt_month'] = train_df['item_cnt_month'].fillna(0)
df.isna().sum()

***Monthly mean feature***

In [ ]:
# Find monthly mean
shop_item_mean = train_df[['shop_id', 'item_id', 'item_cnt_month']].groupby(['shop_id', 'item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_mean = shop_item_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

Creating a feature for the mean of item sales count of each month 

In [ ]:
shop_item_mean.head()

In [ ]:
# Add mean to training set
train_df = pd.merge(train_df, shop_item_mean, how='left', on=['shop_id','item_id'])
train_df.head()

In [ ]:
train_df['date'] = pd.to_datetime(train_df[['year', 'month']].assign(DAY=1)).apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
# Add last month
shop_prev_month = train_df[train_df['date_block_num']==33][['shop_id', 'item_id', 'item_cnt_month']]
shop_prev_month = shop_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})

Adding sales count for the prior month

In [ ]:
shop_prev_month.head()

In [ ]:
# Add previous month feature to training set
train_df = pd.merge(train_df, shop_prev_month, how='left', on=['shop_id', 'item_id']).fillna(0.)

Adding the feature we just made to the training set

In [ ]:
train_df.head()

In [ ]:
# Add all item features
train_df = pd.merge(train_df, items.drop(columns=['item_category_id']), how='left', on='item_id')

Merging the items dataframe into the our current training dataframe 

In [ ]:
# Adding item category features
train_df = pd.merge(train_df, item_categs, how='left', on='item_category_id')

Merging the item cetegory dataframe into the our current training dataframe 

In [ ]:
# Adding shop feature
train_df = pd.merge(train_df, shops, how='left', on='shop_id')

Merging the shops dataframe into the our current training dataframe 

In [ ]:
train_df.head()

The resulting training data after the merge

In [ ]:
test['month']=11
test['year']=2015
test['date_block_num']=34

Adding the prediction value to testing data

In [ ]:
# Adding mean feature
test = pd.merge(test, shop_item_mean, how='left', on=['shop_id', 'item_id']).fillna(0.)
# Adding previous month feature
test = pd.merge(test, shop_prev_month, how='left', on=['shop_id', 'item_id']).fillna(0.)
# Add all item features
test = pd.merge(test, items, how='left', on='item_id')
# Adding item category features
test = pd.merge(test, item_categs, how='left', on='item_category_id')
# Adding shop feature
test = pd.merge(test, shops, how='left', on='shop_id')

Takes the same features we created for the training data and adds them to our teting data

In [ ]:
test['item_cnt_month']=0.

---  
***Some label encoding for the names of items***  
May want to write csv before this and do further encoding in the preprocessing notebook  

In [ ]:
# Label encoding
for c in ['shop_name', 'item_name', 'item_category_name']:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[c].unique())+list(test[c].unique()))
    train_df[c]=lbl.transform(train_df[c].astype(str))
    test[c]=lbl.transform(test[c].astype(str))
    print(c)

---  
---  
### Building a baseline model using Extra Trees Regressor from sci-kit learn

In [ ]:
# Train and predict using random forest 
col = [c for c in train_df.columns if c not in ['item_cnt_month']]
x1 = train_df[train_df['date_block_num']<33]
y1 = np.log1p(x1['item_cnt_month'].clip(0.,20.))
x1 = x1[col]
x2 = train_df[train_df['date_block_num'] == 33]
y2 = np.log1p(x2['item_cnt_month'].clip(0, 20.))
x2 = x2[col]

Creating a method to test our model without having to submit by using the most resent month as a hypothetical future month and then using the values of that month to validate the effectivness of the model

In [ ]:
reg = ensemble.ExtraTreesRegressor(n_estimators=30, n_jobs=-1, max_depth=20, random_state=18)
reg.fit(x1,y1)
print('RMSE value is :', np.sqrt(metrics.mean_squared_error(y2.clip(0., 20.), reg.predict(x2).clip(0., 20.))))

Since the competition grades models based upon the RMSE metric we will use that to evaluate our model so that we are optimizing in that direction

In [ ]:
reg.fit(train_df[col], train_df['item_cnt_month'].clip(0., 20.))
test['item_cnt_month'] = reg.predict(test[col]).clip(0., 20.)

In [ ]:
test[['ID', 'item_cnt_month']].to_csv('submission_files/submit.csv', index=False)

Writing the predictions to a csv file

---  
***Writing training and testing data into csv files***

In [ ]:
test.drop(columns=['item_cnt_month']).to_csv('wrangled_data/testing_dataframe.csv', index=False)
train_df.to_csv('wrangled_data/training_dataframe.csv', index=False)